In [ ]:
# Restart the runtime
# !kill -9 -1

In [3]:
from google.colab import drive  # 40s

drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
# 1. Import Libraries 5s
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold

In [5]:
# 2. Dataset Loading

dataset_path = "/content/drive/MyDrive/Projects/Plant Disease Prediction/a_novel_groundnut_leaf_dataset"
# train_dir = os.path.join(dataset_path, "train")
# test_dir = os.path.join(dataset_path, "test")

# Get the class names (disease names)
class_names = os.listdir(dataset_path)
num_classes = len(class_names)
print(f"Number of classes: {num_classes}")
print(f"Class names: {class_names}")

Number of classes: 5
Class names: ['RUST', 'LEAF SPOT (EARLY AND LATE)', 'HEALTHY', 'ALTERNARIA LEAF SPOT', 'ROSETTE']


In [6]:
import os

def count_images_in_subfolders(folder_path):
    image_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".gif"}
    folder_image_counts = {}

    # Loop through each subfolder inside the given folder
    for sub_folder in os.listdir(folder_path):
        sub_folder_path = os.path.join(folder_path, sub_folder)

        if os.path.isdir(sub_folder_path):  # Ensure it's a folder
            image_count = sum(1 for file in os.listdir(sub_folder_path)
                              if os.path.splitext(file)[1].lower() in image_extensions)
            folder_image_counts[sub_folder] = image_count

    return folder_image_counts

folder_path = "/content/drive/MyDrive/Projects/Plant Disease Prediction/a_novel_groundnut_leaf_dataset"
folder_counts = count_images_in_subfolders(folder_path)
for folder_name, count in folder_counts.items():
    print(f"Folder '{folder_name}' has {count} images.")

Folder 'RUST' has 120 images.
Folder 'LEAF SPOT (EARLY AND LATE)' has 450 images.
Folder 'HEALTHY' has 600 images.
Folder 'ALTERNARIA LEAF SPOT' has 450 images.
Folder 'ROSETTE' has 100 images.


In [ ]:
# 3. Data Preprocessing (6m for 2160 imgs)

images = []
labels = []

for class_name in class_names:
    class_dir = os.path.join(dataset_path, class_name)
    for filename in os.listdir(class_dir):
        if filename.endswith((".jpg", ".png", ".jpeg")):
            img_path = os.path.join(class_dir, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (224, 224))

                # height, width = img.shape[:2]
                # size = min(height, width)
                # x = (width - size) // 2
                # y = (height - size) // 2
                # img = img[y : y + size, x : x + size]

                original_shape = img.shape
                image_2d = img.reshape(-1, 3)
                scaler = MinMaxScaler(feature_range=(0, 1))
                normalized_2d = scaler.fit_transform(image_2d)
                normalized_image = normalized_2d.reshape(original_shape)

                images.append(normalized_image)
                labels.append(class_name)
            else:
                print(f"Error reading image: {img_path}")

images = np.array(images)
labels = np.array(labels)
print(images.shape)
print(labels.shape)

(1720, 224, 224, 3)
(1720,)


In [8]:
# 4. Label Encoding
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
print(labels_encoded)
print(set(labels_encoded))

[4 4 4 ... 3 3 3]
{0, 1, 2, 3, 4}


In [9]:
# 5. Split the dataset
X_train, X_test, y_train, y_test = train_test_split(
    images, labels_encoded, test_size=0.2, random_state=42, stratify=labels_encoded
)
X_val, X_test, y_val, y_test = train_test_split(
    X_test, y_test, test_size=0.5, random_state=42, stratify=y_test
)

print("Train data shape:", X_train.shape)
print("Validation data shape:", X_val.shape)
print("Test data shape:", X_test.shape)

Train data shape: (1376, 224, 224, 3)
Validation data shape: (172, 224, 224, 3)
Test data shape: (172, 224, 224, 3)


In [10]:
# 3. Data Augmentation (Important for small datasets):
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
)

datagen.fit(X_train)  # Fit the datagen on the training data

In [12]:
def create_model():
  model = tf.keras.models.Sequential(   # 3s
      [
          tf.keras.layers.Conv2D(
              32, (3, 3), activation="relu", input_shape=(224, 224, 3)
          ),
          tf.keras.layers.MaxPooling2D((2, 2)),
          tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
          tf.keras.layers.MaxPooling2D((2, 2)),
          tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
          tf.keras.layers.MaxPooling2D((2, 2)),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(128, activation="relu"),
          tf.keras.layers.Dropout(0.5),
          tf.keras.layers.Dense(num_classes, activation="softmax"),
      ]
  )

  model.compile(
      optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
  )
  return model

In [13]:
# 7. Model Training # 1m
epochs = 50      # 30
batch_size = 42  # 32

model = create_model()
history = model.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_val, y_val),
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 18s 264ms/step - accuracy: 0.2962 - loss: 1.7399 - val_accuracy: 0.4477 - val_loss: 1.2374
Epoch 2/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step - accuracy: 0.4696 - loss: 1.2019 - val_accuracy: 0.5640 - val_loss: 1.0037
Epoch 3/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - accuracy: 0.5693 - loss: 1.0128 - val_accuracy: 0.6453 - val_loss: 0.8342
Epoch 4/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - accuracy: 0.6432 - loss: 0.8364 - val_accuracy: 0.6977 - val_loss: 0.7562
Epoch 5/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - accuracy: 0.6932 - loss: 0.7026 - val_accuracy: 0.6744 - val_loss: 0.7692
Epoch 6/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - accuracy: 0.7759 - loss: 0.5842 - val_accuracy: 0.7674 - val_loss: 0.6664
Epoch 7/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - accuracy: 0.8270 - loss: 0.4487 - val_accuracy: 0.7616 - val_loss: 0.6434
Epoch 8/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - accuracy: 0.8851 - loss: 0.3594 - val_accuracy: 0.7849 -

In [14]:
# 8. Model Evaluation:

model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss * 100}")
print(f"Test Accuracy: {accuracy * 100}")

Test Loss: 116.73346757888794
Test Accuracy: 83.72092843055725


In [15]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

y_pred = model.predict(X_test)

y_pred_classes = y_pred.argmax(axis=-1)
y_true = y_test

precision = precision_score(y_true, y_pred_classes, average='weighted')  # For multi-class classification
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')
accuracy = accuracy_score(y_true, y_pred_classes)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"Accuracy: {accuracy * 100:.2f}%")


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 110ms/step
Precision: 0.8380
Recall: 0.8372
F1-Score: 0.8330
Accuracy: 83.72%


In [ ]:
def perform_cross_validation(X, y, k=5, ): #X is the images and y is the labels
    best_model_path="/content/drive/MyDrive//Projects/Plant Disease Prediction/saved_models/5-fold-best-model.keras"
    final_model_path="/content/drive/MyDrive//Projects/Plant Disease Prediction/saved_models/5-fold-final-model.keras"

    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    cv_scores = []
    best_accuracy = 0
    best_model = None

    for fold, (train_index, val_index) in enumerate(skf.split(X, y)):  # Use y directly
        print(f"Fold {fold+1}")

        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]  # Use y directly

        num_classes = len(np.unique(y))  # Get num_classes from y
        model = create_model(num_classes)

        model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), verbose=0)

        _, accuracy = model.evaluate(X_val, y_val, verbose=0)
        cv_scores.append(accuracy)
        print(f"Fold {fold+1} Accuracy: {accuracy}")

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = model

    print(f"Mean Cross-Validation Accuracy: {np.mean(cv_scores)}")

    if best_model:
        print(f"Saving best model to: {best_model_path}")
        best_model.save(best_model_path)

    final_model = create_model(num_classes)
    final_model.fit(X, y, epochs=10, batch_size=32, verbose=0)  # Use y directly

    print(f"Saving final model to: {final_model_path}")
    final_model.save(final_model_path)

    return np.mean(cv_scores)

num_classes = len(np.unique(y))  # Get num_classes *after* loading data
perform_cross_validation(images, labels_encoded)

Fold 1


In [19]:
# from tensorflow.keras.preprocessing import image

def preprocess_image(img_path):
    # Load image
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))  # Resize image to (224, 224)
    height, width = img.shape[:2]
    size = min(height, width)
    x = (width - size) // 2
    y = (height - size) // 2
    img = img[y : y + size, x : x + size]  # Crop image to square

    # Normalize the image
    original_shape = img.shape
    image_2d = img.reshape(-1, 3)
    scaler = MinMaxScaler(feature_range=(0, 1))
    normalized_2d = scaler.fit_transform(image_2d)
    normalized_image = normalized_2d.reshape(original_shape)

    # Add an extra dimension for batch size (1 image in this case)
    normalized_image = np.expand_dims(normalized_image, axis=0)

    return normalized_image

# Path to the input image
# img_path = "/content/drive/MyDrive/Projects/Plant Disease Prediction/Groundnut_Leaf_dataset/test/nutriti=on_deficiency_1/dr_4_9961.jpg"
# img_path = "/content/drive/MyDrive/Projects/Plant Disease Prediction/Groundnut_Leaf_dataset/test/rust_1/IMG_9015.jpg"
# img_path = "/content/drive/MyDrive/Projects/Plant Disease Prediction/Groundnut_Leaf_dataset/test/healthy_leaf_1/dr_4_1013.jpg"
img_path = "/content/drive/MyDrive/Projects/Plant Disease Prediction/Groundnut_Leaf_dataset/test/late_leaf_spot_1/dr_4_9915.jpg"
# img_path = "/content/drive/MyDrive/Projects/Plant Disease Prediction/unknown_predict_image/early_leaf_spot.jpg"

# Preprocess the image
input_img = preprocess_image(img_path)

# Make prediction
predictions = model.predict(input_img)
predicted_class = np.argmax(predictions)  # Get class with highest probability
confidence = np.max(predictions)  # Get confidence score

print(f"Predicted Class: {predicted_class}, Confidence: {confidence * 100:.2f}%")

import pickle

# Load label encoder
le_path = "/content/drive/MyDrive/Projects/Plant Disease Prediction/saved_models/label_encoder.pkl"
with open(le_path, "rb") as f:
    le = pickle.load(f)

# Get the disease name
predicted_label = le.inverse_transform([predicted_class])[0]
print(f"Predicted Disease: {predicted_label}, Confidence: {confidence * 100:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 685ms/step
Predicted Class: 2, Confidence: 94.54%
Predicted Disease: LEAF SPOT (EARLY AND LATE), Confidence: 94.54%


In [20]:
# 7. Saving the Model and Label Encoder:

import pickle
models_dir = "/content/drive/MyDrive//Projects/Plant Disease Prediction/saved_models"
os.makedirs(models_dir, exist_ok=True)

model_save_path = os.path.join(models_dir, "84acc_groundnut_disease_model.keras")
model.save(model_save_path)

le_save_path = os.path.join(models_dir, "84acc_label_encoder.pkl")

with open(le_save_path, "wb") as f:
    pickle.dump(le, f)

# print(f"Model saved to: {model_save_path}")
print(f"Label encoder saved to: {le_save_path}")

Label encoder saved to: /content/drive/MyDrive//Projects/Plant Disease Prediction/saved_models/84acc_label_encoder.pkl


In [ ]:
# 9. Plotting Training History

plt.plot(history.history["accuracy"], label="Train Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()